### Required Libraries

In [49]:
# beautifulsoup4      4.9.3
# fake-useragent      0.1.11
# helium              3.0.5
# ipython             7.22.0
# matplotlib          3.3.4
# pandas              1.2.4
# Pillow              8.2.0
# requests            2.25.1
# tqdm                4.59.0

In [1]:
from helium import *
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm as tq
import time
import requests
import os
from fake_useragent import UserAgent

In [35]:
df_category = pd.read_csv('kitchen_2.csv')
df_category.head()

,product_name,product_link_name,link,major_category_name,major_category_link
0,AREA RUGS & FLOORING,AREA RUGS & FLOORING,https://www.canadiantire.ca/en/household-pets/...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
1,ARTIFICIAL FLOWERS & PLANTS,ARTIFICIAL FLOWERS & PLANTS,https://www.canadiantire.ca/en/household-pets/...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
2,CANDLES,CANDLES,https://www.canadiantire.ca/en/household-pets/...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
3,DECORATIVE PILLOWS,DECORATIVE PILLOWS,https://www.canadiantire.ca/en/household-pets/...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....
4,WALL CLOCKS,WALL CLOCKS,https://www.canadiantire.ca/en/household-pets/...,HOME & PETS,https://www.canadiantire.ca/en/household-pets....


In [36]:
df_category.shape

(102, 5)

In [4]:
def update_link(link):
    split = link.split('?') 
    complement = 'https:'
    updated_link = complement + split[0]
    return updated_link

In [37]:
from selenium.webdriver.firefox.options import Options
options = Options()

ua = UserAgent()

import random
randomlist = []
for i in range(0,5):
    n = random.randint(50, 120)
    randomlist.append(n/10)

df_list = []
links_not_fetched = []
for i in tq(range(0, len(df_category[:20]))):
    print(i)
    product_name = df_category['product_name'][i]
    class_name = df_category['product_link_name'][i]
    url = df_category['link'][i]+'?'
    
    print('Start fetching process for {}!'.format(class_name))
    
    if url[-3:] != '_no':
        number_of_pages = 5 
        url_list = [url + 'page={}'.format(i) for i in range(1, number_of_pages+1)]
    else:
        url_list = [url[:-3]]
    
    soup_list = []
    count = 0
    for page in tq(url_list):
        start = time.time()
        user_agent = ua.firefox
        options.add_argument('--user-agent="{}"'.format(user_agent))
        browser = start_firefox(page, headless=True, options=options)
        time.sleep(5)
        press(ESCAPE)
        time.sleep(8)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        checker = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        if len(checker) > 0:
            soup_list.append(soup)
            print('Page fetched: {}.'.format(page))
        elif (len(checker)==0) & (count < 2):
            count += 1
            print('Page not fetched: {}.'.format(page))
        else:
            print('Counter: ', count)
            print('Page not fetched: {}.'.format(page))
            links_not_fetched.append(page)
            print("No more pages avaiable")
            kill_browser()
            sleeping = random.choice(randomlist)
            time.sleep(sleeping)  
            end = time.time()
            time_elapsed = round(end - start, 1)
            print('Time elapsed: {}s'.format(time_elapsed))
            print('\n')
            try:
                kill_browser()
                print('Browser closed.')
            except:
                print('There were no browser open.')
                pass
            break

        try:
            kill_browser()
            print('Browser closed.')
        except:
            print('There were no browser open.')
            pass

        sleeping = random.choice(randomlist)
        time.sleep(sleeping)  
        end = time.time()
        time_elapsed = round(end - start, 1)
        print('Time elapsed: {}s'.format(time_elapsed))
    print('Fetching process for {} finished!'.format(class_name))
    print('Creating dataframe.')
    
    image_link_list = []
    original_name_list = []

    for soup in soup_list: 
        products_list = soup.find_all('div', {'class':'product-tile-srp__image-wrapper'})
        for product in products_list:
            image = product.find('img')
            link = image['src']
            link = update_link(link)
            original_name = image['alt']
            image_link_list.append(link)
            original_name_list.append(original_name)

    df_temp = pd.DataFrame({'original_name': original_name_list, 'image_link': image_link_list})
    df_temp['class_name'] = class_name
    df_temp['product_name'] = product_name
    df_list.append(df_temp)
    print('Dataframe for {} created.\n\n\n\n'.format(class_name))

print('Creating final dataset.')
df_final = pd.concat(df_list)
df_final.drop_duplicates(subset='image_link', inplace=True)
df_final.reset_index(inplace=True)
del df_final['index']
print('Process finished. Printing final dataset.')
df_final

  0%|          | 0/5 [00:00<?, ?it/s]

0
Start fetching process for AREA RUGS & FLOORING!
Page fetched: https://www.canadiantire.ca/en/household-pets/home-decor/area-rugs-flooring.html?adlocation=LIT_Category_Product_AreaRugsAndFlooringCat_en?page=1.
Browser closed.



 20%|██        | 1/5 [00:49<03:17, 49.37s/it]

Time elapsed: 49.4s
Page fetched: https://www.canadiantire.ca/en/household-pets/home-decor/area-rugs-flooring.html?adlocation=LIT_Category_Product_AreaRugsAndFlooringCat_en?page=2.
Browser closed.



 40%|████      | 2/5 [01:35<02:22, 47.42s/it]

...


 80%|████████  | 4/5 [03:36<00:52, 52.66s/it]

Time elapsed: 48.6s
Page fetched: https://www.canadiantire.ca/en/lighting/fluorescent-light-fixtures.html?adlocation=LIT_Lighting_FlourescentLights_en?page=5.
Browser closed.



100%|██████████| 20/20 [1:22:57<00:00, 248.88s/it]

Time elapsed: 48.4s
Fetching process for FLUORESCENT LIGHT FIXTURES finished!
Creating dataframe.
Dataframe for FLUORESCENT LIGHT FIXTURES created.




Creating final dataset.
Process finished. Printing final dataset.


,original_name,image_link,class_name,product_name
0,"Anti-Fatigue Kitchen Mat, 20-in x 38-in",https://canadiantire.scene7.com/is/image/Canad...,AREA RUGS & FLOORING,AREA RUGS & FLOORING
1,Concord Mat with Bonus Platinum Mat,https://canadiantire.scene7.com/is/image/Canad...,AREA RUGS & FLOORING,AREA RUGS & FLOORING
2,"Concord Mat, 22-in x 36-in",https://canadiantire.scene7.com/is/image/Canad...,AREA RUGS & FLOORING,AREA RUGS & FLOORING
3,Platinum Charcoal Mat,https://canadiantire.scene7.com/is/image/Canad...,AREA RUGS & FLOORING,AREA RUGS & FLOORING
4,Multy Home Blue Poppy Floor Mat,https://canadiantire.scene7.com/is/image/Canad...,AREA RUGS & FLOORING,AREA RUGS & FLOORING
...,...,...,...,...
946,"Fluorescent Light Reflector with Cord, 48-in",https://canadiantire.scene7.com/is/image/Canad...,FLUORESCENT LIGHT FIXTURES,FLUORESCENT LIGHT FIXTURES
947,"NOMA LED Shop Light with Motion Sensor, 4-ft",https://canadiantire.scene7.com/is/image/Canad...,FLUORESCENT LIGHT FIXTURES,FLUORESCENT LIGHT FIXTURES
948,"NOMA Single LED Strip Light, 2-ft",https://canadiantire.scene7.com/is/image/Canad...,FLUORESCENT LIGHT FIXTURES,FLUORESCENT LIGHT FIXTURES
949,"Canarm 59W Fluorescent Replacement Ballast, 96-in",https://canadiantire.scene7.com/is/image/Canad...,FLUORESCENT LIGHT FIXTURES,FLUORESCENT LIGHT FIXTURES


In [38]:
df_final.to_csv('kitchen_dataset.csv', index=False)